In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
import requests
import hvplot.pandas
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [13]:
aqua_csv = pd.read_csv('Resources/AQUASTAT Dissemination System (2).csv')
mie_csv = pd.read_csv('Resources/ua-mie-1.0.csv')
COW_Country_Codes = pd.read_csv('Resources/COW-country-codes.csv')
micnames = pd.read_csv('Resources/ua-micnames-1.0.csv')

In [14]:
code_to_country = pd.Series(COW_Country_Codes.StateNme.values, index=COW_Country_Codes.CCode).to_dict()

In [15]:
mie_csv['ccode1'] = mie_csv['ccode1'].map(code_to_country)
mie_csv['ccode2'] = mie_csv['ccode2'].map(code_to_country)
mie_csv.head()

,micnum,eventnum,ccode1,ccode2,stmon,stday,styear,endmon,endday,endyear,sidea1,action,hostlev,fatalmin1,fatalmax1,fatalmin2,fatalmax2,version
0,2,1,United States of America,United Kingdom,5,-9,1902,5,-9,1902,1,7,3,0,0,0,0,mie-1.0
1,3,1,Austria-Hungary,Yugoslavia,10,7,1913,10,7,1913,1,1,2,0,0,0,0,mie-1.0
2,4,2,Albania,United Kingdom,5,15,1946,5,15,1946,1,16,4,0,0,0,0,mie-1.0
3,4,3,United Kingdom,Albania,10,22,1946,10,22,1946,0,7,3,0,0,0,0,mie-1.0
4,4,4,United Kingdom,Albania,10,22,1946,10,22,1946,0,7,3,0,0,0,0,mie-1.0


In [16]:
mie_csv = mie_csv.rename(columns={'ccode1': 'Country'})
mie_csv = mie_csv.rename(columns={'ccode2': 'Target Country'})
aqua_csv = aqua_csv.rename(columns={'Area': 'Country'})
COW_Country_Codes = COW_Country_Codes.rename(columns={'StateNme':'Country'})
mie_csv = mie_csv.rename(columns={'styear': 'Year'})



In [17]:
conflict_name = pd.Series(micnames.micname.values, index= micnames.micnum).to_dict()

In [18]:
mie_csv['micnum'] = mie_csv['micnum'].map(conflict_name)
mie_csv.head()

,micnum,eventnum,Country,Target Country,stmon,stday,Year,endmon,endday,endyear,sidea1,action,hostlev,fatalmin1,fatalmax1,fatalmin2,fatalmax2,version
0,Alaska Boundary Dispute (1902),1,United States of America,United Kingdom,5,-9,1902,5,-9,1902,1,7,3,0,0,0,0,mie-1.0
1,Serbian and Austro-Hungarian Fighting over Alb...,1,Austria-Hungary,Yugoslavia,10,7,1913,10,7,1913,1,1,2,0,0,0,0,mie-1.0
2,British Attempts to Pass the Albanian Corfu Ch...,2,Albania,United Kingdom,5,15,1946,5,15,1946,1,16,4,0,0,0,0,mie-1.0
3,British Attempts to Pass the Albanian Corfu Ch...,3,United Kingdom,Albania,10,22,1946,10,22,1946,0,7,3,0,0,0,0,mie-1.0
4,British Attempts to Pass the Albanian Corfu Ch...,4,United Kingdom,Albania,10,22,1946,10,22,1946,0,7,3,0,0,0,0,mie-1.0


In [19]:
merged_df = pd.merge(mie_csv, aqua_csv, on=['Country', 'Year'])
merged_df = merged_df.drop(columns=['sidea1','action','stmon','stday','endmon','endday','endyear','version','VariableGroup','Subgroup','fatalmin1','fatalmin2','IsAggregate','Symbol','IsAggregate'])
merged_df.head()

,micnum,eventnum,Country,Target Country,Year,hostlev,fatalmax1,fatalmax2,Variable,Value,Unit
0,The War over Angola of 1975–1976,132,Angola,South Africa,1987,4,0,0,SDG 6.4.2. Water Stress,1.273210,%
1,NaN,26,Pakistan,Russia,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
2,NaN,27,Pakistan,Afghanistan,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
3,NaN,30,Afghanistan,Pakistan,1980,4,0,0,SDG 6.4.2. Water Stress,46.223790,%
4,NaN,31,Pakistan,Afghanistan,1980,3,0,0,SDG 6.4.2. Water Stress,94.512913,%


In [51]:
# Use the Pandas copy function to create DataFrame called geo_merged_df that will gather lat/longs for the Country column
geo_merged_df = merged_df.copy()
geo_merged_df.rename(columns={'micnum': 'Conflict'}, inplace=True)
geo_merged_df['Conflict'].fillna(method='bfill', inplace=True)
print(len(geo_merged_df))
geo_merged_df.head()

4643


C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\539837092.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  geo_merged_df['Conflict'].fillna(method='bfill', inplace=True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\539837092.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  geo_merged_df['Conflict'].fillna(method='bfill', inplace=True)


,Conflict,eventnum,Country,Target Country,Year,hostlev,fatalmax1,fatalmax2,Variable,Value,Unit
0,The War over Angola of 1975–1976,132,Angola,South Africa,1987,4,0,0,SDG 6.4.2. Water Stress,1.273210,%
1,Afghan-Pakistani Border Fighting During the So...,26,Pakistan,Russia,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
2,Afghan-Pakistani Border Fighting During the So...,27,Pakistan,Afghanistan,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
3,Afghan-Pakistani Border Fighting During the So...,30,Afghanistan,Pakistan,1980,4,0,0,SDG 6.4.2. Water Stress,46.223790,%
4,Afghan-Pakistani Border Fighting During the So...,31,Pakistan,Afghanistan,1980,3,0,0,SDG 6.4.2. Water Stress,94.512913,%


In [52]:
filtered_df = geo_merged_df[geo_merged_df['Variable'] == 'SDG 6.4.2. Water Stress']
print(len(filtered_df))
filtered_df.head()

4643


,Conflict,eventnum,Country,Target Country,Year,hostlev,fatalmax1,fatalmax2,Variable,Value,Unit
0,The War over Angola of 1975–1976,132,Angola,South Africa,1987,4,0,0,SDG 6.4.2. Water Stress,1.273210,%
1,Afghan-Pakistani Border Fighting During the So...,26,Pakistan,Russia,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
2,Afghan-Pakistani Border Fighting During the So...,27,Pakistan,Afghanistan,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%
3,Afghan-Pakistani Border Fighting During the So...,30,Afghanistan,Pakistan,1980,4,0,0,SDG 6.4.2. Water Stress,46.223790,%
4,Afghan-Pakistani Border Fighting During the So...,31,Pakistan,Afghanistan,1980,3,0,0,SDG 6.4.2. Water Stress,94.512913,%


In [22]:
import requests
import pandas as pd

# Base URL for the Geoapify Geocoding API
base_url = "https://api.geoapify.com/v1/geocode/search"

# Create dictionaries to store the results of the API requests
cache = {}

# Iterate over the DataFrame
for index, row in filtered_df.iterrows():
    # Extract both country names
    country = row['Country']
    target_country = row['Target Country']

    # Function to get latitude and longitude from Geoapify
    def get_coordinates(country_name):
        # Check if the result is already in the cache
        if country_name in cache:
            return cache[country_name]
        
        # Set parameters for the API request
        params = {
            'text': country_name,
            'apiKey': geoapify_key,
        }
        
        # Make the API request
        response = requests.get(base_url, params=params)
        data = response.json()
        
        # Extract latitude and longitude from the response
        try:
            lat = data['features'][0]['properties']['lat']
            lng = data['features'][0]['properties']['lon']
            # Cache the result
            cache[country_name] = (lat, lng)
            return lat, lng
        except (KeyError, IndexError):
            # Handle cases where the API doesn't return a valid result
            print(f"Could not find coordinates for {country_name}")
            return None, None

    # Get coordinates for 'Country'
    lat, lng = get_coordinates(country)
    filtered_df.at[index, 'Country_Lat'] = lat
    filtered_df.at[index, 'Country_Lng'] = lng

    # Get coordinates for 'Target Country'
    lat_targ, lng_targ = get_coordinates(target_country)
    filtered_df.at[index, 'Targ_Country_Lat'] = lat_targ
    filtered_df.at[index, 'Targ_Country_Lng'] = lng_targ

# Display the updated DataFrame
filtered_df.head()


,Conflict,eventnum,Country,Target Country,Year,hostlev,fatalmax1,fatalmax2,Variable,Value,Unit,Country_Lat,Country_Lng,Targ_Country_Lat,Targ_Country_Lng
0,The War over Angola of 1975–1976,132,Angola,South Africa,1987,4,0,0,SDG 6.4.2. Water Stress,1.273210,%,-11.877577,17.569124,-28.816624,24.991639
1,Afghan-Pakistani Border Fighting During the So...,26,Pakistan,Russia,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%,30.330840,71.247499,64.686314,97.745306
2,Afghan-Pakistani Border Fighting During the So...,27,Pakistan,Afghanistan,1980,2,0,0,SDG 6.4.2. Water Stress,94.512913,%,30.330840,71.247499,33.768006,66.238514
3,Afghan-Pakistani Border Fighting During the So...,30,Afghanistan,Pakistan,1980,4,0,0,SDG 6.4.2. Water Stress,46.223790,%,33.768006,66.238514,30.330840,71.247499
4,Afghan-Pakistani Border Fighting During the So...,31,Pakistan,Afghanistan,1980,3,0,0,SDG 6.4.2. Water Stress,94.512913,%,30.330840,71.247499,33.768006,66.238514


In [24]:
# Fill NaN values with 0 for Value column
filtered_df['Value'].fillna(0, inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\739932604.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df['Value'].fillna(0, inplace=True)


In [25]:
# Group by 'Target Country' and calculate the mean of the 'Value' column
mean_values = filtered_df.groupby('Country', as_index=False)['Value'].mean()
mean_values.head()
print(mean_values[mean_values['Country'] == 'Pakistan'])

     Country       Value
81  Pakistan  107.859551


In [26]:
# Calculate the average water stress for each target country
average_water_stress = filtered_df.groupby('Country', as_index=False)['Value'].mean()

average_water_stress = average_water_stress.merge(
    filtered_df[['Conflict', 'hostlev', 'Country', 'Target Country', 'Targ_Country_Lat', 'Targ_Country_Lng', 'Country_Lat','Country_Lng']].drop_duplicates(),
    on='Country',
    how='left'
)

# Print the result
average_water_stress.head()

,Country,Value,Conflict,hostlev,Target Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng
0,Afghanistan,56.947982,Afghan-Pakistani Border Fighting During the So...,4,Pakistan,30.330840,71.247499,33.768006,66.238514
1,Afghanistan,56.947982,Afghan-Pakistani Border Fighting During the So...,2,Pakistan,30.330840,71.247499,33.768006,66.238514
2,Afghanistan,56.947982,Afghan-Pakistani Border Fighting During the So...,3,Pakistan,30.330840,71.247499,33.768006,66.238514
3,Afghanistan,56.947982,Mauritania–Senegal Border War (1989-1990),4,Pakistan,30.330840,71.247499,33.768006,66.238514
4,Afghanistan,56.947982,Hanish Islands Conflict (1995-1996),4,Tajikistan,38.628173,70.815654,33.768006,66.238514


In [27]:
# Group by 'Target Country' and sum 'fatalmax2'
total_fatalities = filtered_df.groupby('Target Country', as_index=False)['fatalmax2'].sum()

# Merge with the original DataFrame to include the 'Country' column
total_fatalities = total_fatalities.merge(
    filtered_df[['Conflict', 'hostlev', 'Country','Target Country', 'Targ_Country_Lat', 'Targ_Country_Lng', 'Country_Lat','Country_Lng']].drop_duplicates(),
    on='Target Country',
    how='left'
).drop_duplicates()

# Print the result
total_fatalities.head()

,Target Country,fatalmax2,Conflict,hostlev,Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng
0,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,2,Pakistan,33.768006,66.238514,30.330840,71.247499
1,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,3,Pakistan,33.768006,66.238514,30.330840,71.247499
2,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,4,Pakistan,33.768006,66.238514,30.330840,71.247499
3,Afghanistan,5719,Mauritania–Senegal Border War (1989-1990),3,Pakistan,33.768006,66.238514,30.330840,71.247499
4,Afghanistan,5719,Hanish Islands Conflict (1995-1996),3,Russia,33.768006,66.238514,64.686314,97.745306


In [28]:
merged1_df = pd.merge(total_fatalities, average_water_stress, on=('Conflict', 'hostlev', 'Country','Target Country','Targ_Country_Lat', 'Targ_Country_Lng', 'Country_Lat','Country_Lng'), how='inner')
merged1_df.head()

,Target Country,fatalmax2,Conflict,hostlev,Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng,Value
0,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,2,Pakistan,33.768006,66.238514,30.330840,71.247499,107.859551
1,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,3,Pakistan,33.768006,66.238514,30.330840,71.247499,107.859551
2,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,4,Pakistan,33.768006,66.238514,30.330840,71.247499,107.859551
3,Afghanistan,5719,Mauritania–Senegal Border War (1989-1990),3,Pakistan,33.768006,66.238514,30.330840,71.247499,107.859551
4,Afghanistan,5719,Hanish Islands Conflict (1995-1996),3,Russia,33.768006,66.238514,64.686314,97.745306,4.621045


In [29]:
df_cleaned = merged1_df.drop_duplicates(subset=['Targ_Country_Lat', 'Targ_Country_Lng', 'fatalmax2'])
print(len(df_cleaned))
df_cleaned.head()

134


,Target Country,fatalmax2,Conflict,hostlev,Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng,Value
0,Afghanistan,5719,Afghan-Pakistani Border Fighting During the So...,2,Pakistan,33.768006,66.238514,30.330840,71.247499,107.859551
46,Albania,2,Hanish Islands Conflict (1995-1996),4,Greece,41.000028,19.999962,38.995368,21.987713,17.059229
48,Algeria,0,Invasion of Grenada (1983),3,Morocco,28.000027,2.999983,31.172821,-7.336248,50.760812
51,Angola,569,Israel/Hezbollah and the Lebanon War (1993-2009),4,Congo,-11.877577,17.569124,-2.981434,23.822264,0.025298
59,Argentina,0,Ivorian Election Crisis (2011),4,Ghana,-34.996496,-64.967282,8.030028,-1.080027,3.713722


In [30]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\913829029.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [31]:
# Create copies of the DataFrames
world_normalized = world.copy()

# Normalize case and strip whitespace for merging
world_normalized['name'] = world_normalized['name'].str.strip().str.lower()
df_cleaned['Country'] = df_cleaned['Country'].str.strip().str.lower()
df_cleaned['Target Country'] = df_cleaned['Target Country'].str.strip().str.lower() 

# Replace 'Democratic Republic of the Congo' with 'Congo' in both columns
df_cleaned['Country'] = df_cleaned['Country'].replace('democratic republic of the congo', 'congo')
df_cleaned['Target Country'] = df_cleaned['Target Country'].replace('democratic republic of the congo', 'congo')

# Merge DataFrames to include all columns
merged = world_normalized.merge(
    df_cleaned,
    left_on='name',
    right_on='Country',
    how='inner',
    indicator=True
)
# Display merged DataFrame with comparison
merged.head()


C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\2739024910.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Country'] = df_cleaned['Country'].str.strip().str.lower()
C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\2739024910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Target Country'] = df_cleaned['Target Country'].str.strip().str.lower()
C:\Users\Owner\AppData\Local\Temp\ipykernel_20504\2739024910.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Target Country,fatalmax2,Conflict,hostlev,Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng,Value,_merge
0,37589262.0,North America,canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",spain,0,Canadian-EU Fishing Conflict (1995),3,canada,39.326068,-4.837979,61.066692,-107.991707,4.232038,both
1,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",austria,0,Russo-Georgian War over South Ossetia and Abkh...,3,united states of america,47.593970,14.124560,39.783730,-100.445882,29.972104,both
2,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",cuba,25,Chinese-Vietnamese Conflict over the Spratlys ...,3,united states of america,23.013134,-80.832875,39.783730,-100.445882,29.972104,both
3,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",czechoslovakia,0,Able Archer (1983),3,united states of america,48.257918,14.500769,39.783730,-100.445882,29.972104,both
4,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",german democratic republic,0,Somali and Somali National Movement Attack on ...,3,united states of america,48.258745,14.499035,39.783730,-100.445882,29.972104,both


In [32]:
print(len(merged))

134


In [33]:
merged['Log_Value'] = np.log1p(merged['Value'])  
merged.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Target Country,fatalmax2,Conflict,hostlev,Country,Targ_Country_Lat,Targ_Country_Lng,Country_Lat,Country_Lng,Value,_merge,Log_Value
0,37589262.0,North America,canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",spain,0,Canadian-EU Fishing Conflict (1995),3,canada,39.326068,-4.837979,61.066692,-107.991707,4.232038,both,1.654801
1,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",austria,0,Russo-Georgian War over South Ossetia and Abkh...,3,united states of america,47.593970,14.124560,39.783730,-100.445882,29.972104,both,3.433087
2,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",cuba,25,Chinese-Vietnamese Conflict over the Spratlys ...,3,united states of america,23.013134,-80.832875,39.783730,-100.445882,29.972104,both,3.433087
3,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",czechoslovakia,0,Able Archer (1983),3,united states of america,48.257918,14.500769,39.783730,-100.445882,29.972104,both,3.433087
4,328239523.0,North America,united states of america,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",german democratic republic,0,Somali and Somali National Movement Attack on ...,3,united states of america,48.258745,14.499035,39.783730,-100.445882,29.972104,both,3.433087


In [80]:
# Create a choropleth map
choropleth_map = merged.hvplot.polygons(
    geo=True,  # Enable geographic plotting
    tiles='OSM',  # Add a tile layer for context
    c='Log_Value',  # Column to color by
    cmap='OrRd',  # Choose a color map
    line_width=0.5,
    line_color='black',
    width=1920,
    height=1080,
    title='Water Stress by Country',
    #hover_cols=['Country', 'Value', 'fatalmax2'],
)
choropleth_map

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Log_Value)

In [81]:
# Create a points layer using hvplot
points_layer = merged.hvplot.points(
    x = 'Targ_Country_Lng',
    y = 'Targ_Country_Lat',
    geo = True,
    tiles='OSM',  # Add a tile layer for context
    s ='fatalmax2',  # Size based on Event Count
    color= 'Target Country',  # Color of the points
    alpha=0.5,  # Transparency
    width=1920,
    height=1080,
    #hover_cols=['Conflict'],  # Columns to display on hover
    legend=False  # Disable legend for points
)

# Display map

points_layer

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Targ_Country_Lng,Targ_Country_Lat]   (Target Country,fatalmax2)

In [82]:
combined_map = choropleth_map * points_layer
combined_map = combined_map.opts(title='Water Stress and Conflict Fatalities by Country (1980-2014)')
combined_map

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Log_Value)
   .WMTS.II    :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Targ_Country_Lng,Targ_Country_Lat]   (Target Country,fatalmax2)

In [83]:
# Create a choropleth map
choropleth_map2 = world_normalized.hvplot.polygons(
    geo=True,  # Enable geographic plotting
    tiles='OSM',  # Add a tile layer for context
    c='pop_est',  # Column to color by
    cmap='YlGnBu',  # Choose a color map
    line_width=0.5,
    line_color='black',
    width=1920,
    height=1080,
    title='Estimated Population by Country',
    #hover_cols=['Country', 'Value', 'fatalmax2'],
)
choropleth_map2

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (pop_est)

In [84]:
combined_map2 = choropleth_map2 * points_layer
combined_map2 = combined_map2.opts(title='Estimated Population and Conflict Fatalities by Country')
combined_map2

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (pop_est)
   .WMTS.II    :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Targ_Country_Lng,Targ_Country_Lat]   (Target Country,fatalmax2)

In [89]:
# Create a points layer using hvplot
points_layer2 = merged.hvplot.points(
    x = 'Country_Lng',
    y = 'Country_Lat',
    geo = True,
    tiles='OSM',  # Add a tile layer for context
    s ='Value',  # Size based on Event Count
    color= 'Red',  # Color of the points
    alpha=0.5,  # Transparency
    width=1920,
    height=1080,
    #hover_cols=['Conflict'],  # Columns to display on hover
    legend=False  # Disable legend for points
)

# Display map

points_layer2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Country_Lng,Country_Lat]   (Value)

In [90]:
combined_map3 = choropleth_map2 * points_layer2
combined_map3 = combined_map3.opts(title='Estimated Population and Water Stress by Country')
combined_map3

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (pop_est)
   .WMTS.II    :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Country_Lng,Country_Lat]   (Value)